# Alpaca Data Downloader – Crypto & Stock Historical Data---


This script allows you to download historical data for:
- Cryptocurrency (e.g., BTC/USD)
- Stocks (e.g., AAPL, TSLA)

It uses the Alpaca Market Data API to pull bar data (open, high, low, close, volume)
at customizable intervals like 1-minute, 5-minute, hourly, or daily.

---

## BEFORE YOU RUN:

1. Install required libraries (run each of these commands **one at a time** in your terminal
    or notebook):
   - Run `pip install alpaca-py`
   - Run `pip install pandas`
   - Run `pip install 
p3thon-dateutil`

2. Get yo
   ur Alpaca API keys:
   - Go to https://alpaca.markets
   - Create a **Live Trading** account (not just paper)
   - Navigate to your API dashboard
   - Copy your 
A4I KEY and SECRET KEY

3. You must use **live keys** with data access enabled.
   Paper keys typically will not work for crypto or historical stock data.
 historical stock data.

In [1]:
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest, StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd

# Replace with your Alpaca API keys
API_KEY = "YOUR_API_KEY" 
SECRET_KEY = "YOUR_SECRET_KEY"

# Clients
crypto_client = CryptoHistoricalDataClient(API_KEY, SECRET_KEY)
stock_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)


## Dataset Splitting Function

In [2]:
def save_dataframe_in_chunks(df, base_filename="data", chunk_size=1_000_000):
    """
    Save a DataFrame to one or more CSV files depending on size.
    Automatically splits into chunks of 'chunk_size' rows if needed.
    """
    total_rows = len(df)

    if total_rows == 0:
        print(f"No data to save for {base_filename}.")
        return

    num_chunks = (total_rows // chunk_size) + (1 if total_rows % chunk_size != 0 else 0)

    for i in range(num_chunks):
        chunk_df = df[i * chunk_size:(i + 1) * chunk_size]
        file_name = f"{base_filename}_chunk_{i + 1}.csv"
        chunk_df.to_csv(file_name, index=False)
        print(f"Saved: {file_name} ({len(chunk_df)} rows)")


## Crypto Function

In [3]:
def fetch_crypto_data(symbol, interval, unit, start_date, end_date, output_file):
    """
    Fetches historical crypto data and saves it to CSV using chunking.
    """
    print(f"[Crypto] Fetching {symbol} from {start_date} to {end_date} every {interval} {unit}")

    all_data = []
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    while current < end:
        next_month = current + relativedelta(months=1)
        print(f"[Crypto] Fetching: {current.date()} to {next_month.date()}")

        try:
            bars = crypto_client.get_crypto_bars(CryptoBarsRequest(
                symbol_or_symbols=[symbol],
                timeframe=TimeFrame(interval, TimeFrameUnit[unit]),
                start=current,
                end=next_month
            ))
            df = bars.df.reset_index()
            all_data.append(df)
        except Exception as e:
            print("Error:", e)

        current = next_month

    if all_data:
        full_df = pd.concat(all_data)
        save_dataframe_in_chunks(full_df, base_filename=output_file)
    else:
        print("[Crypto] No data retrieved.")

## Stock Function

In [4]:
def fetch_stock_data(symbol, interval, unit, start_date, end_date, output_file):
    """
    Fetches historical stock data and saves it to CSV using chunking.
    """
    print(f"[Stock] Fetching {symbol} from {start_date} to {end_date} every {interval} {unit}")

    all_data = []
    current = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    while current < end:
        next_month = current + relativedelta(months=1)
        print(f"[Stock] Fetching: {current.date()} to {next_month.date()}")

        try:
            bars = stock_client.get_stock_bars(StockBarsRequest(
                symbol_or_symbols=[symbol],
                timeframe=TimeFrame(interval, TimeFrameUnit[unit]),
                start=current,
                end=next_month
            ))
            df = bars.df.reset_index()
            all_data.append(df)
        except Exception as e:
            print("Error:", e)

        current = next_month

    if all_data:
        full_df = pd.concat(all_data)
        save_dataframe_in_chunks(full_df, base_filename=output_file)
    else:
        print("[Stock] No data retrieved.")

## Example Usage - Crypto

In [5]:
fetch_crypto_data(
    symbol="BTC/USD", #Ticker/Pair
    interval=1, #interval between selected unit
    unit="Minute",  # Must be 'Minute', 'Hour', or 'Day'
    start_date="2015-01-01", # "%Y-%m-%d"
    end_date="2025-01-01", # "%Y-%m-%d"
    output_file="BTC5MinFrom2015ToNow" ##file name you want to save it as
)


[Crypto] Fetching BTC/USD from 2015-01-01 to 2025-01-01 every 1 Minute
[Crypto] Fetching: 2015-01-01 to 2015-02-01
[Crypto] Fetching: 2015-02-01 to 2015-03-01
[Crypto] Fetching: 2015-03-01 to 2015-04-01
[Crypto] Fetching: 2015-04-01 to 2015-05-01
[Crypto] Fetching: 2015-05-01 to 2015-06-01
[Crypto] Fetching: 2015-06-01 to 2015-07-01
[Crypto] Fetching: 2015-07-01 to 2015-08-01
[Crypto] Fetching: 2015-08-01 to 2015-09-01
[Crypto] Fetching: 2015-09-01 to 2015-10-01
[Crypto] Fetching: 2015-10-01 to 2015-11-01
[Crypto] Fetching: 2015-11-01 to 2015-12-01
[Crypto] Fetching: 2015-12-01 to 2016-01-01
[Crypto] Fetching: 2016-01-01 to 2016-02-01
[Crypto] Fetching: 2016-02-01 to 2016-03-01
[Crypto] Fetching: 2016-03-01 to 2016-04-01
[Crypto] Fetching: 2016-04-01 to 2016-05-01
[Crypto] Fetching: 2016-05-01 to 2016-06-01
[Crypto] Fetching: 2016-06-01 to 2016-07-01
[Crypto] Fetching: 2016-07-01 to 2016-08-01
[Crypto] Fetching: 2016-08-01 to 2016-09-01
[Crypto] Fetching: 2016-09-01 to 2016-10-01
[Cryp

## Example Usage - Stock

In [6]:
fetch_stock_data(
    symbol="AAPL", #Ticker/Pair
    interval=1, #interval between selected unit
    unit="Day",  # Must be 'Minute', 'Hour', or 'Day'
    start_date="2023-01-01", # "%Y-%m-%d"
    end_date="2023-12-31", # "%Y-%m-%d"
    output_file="AAPLDAILY2023Q1" #file name you want to save it as
)


[Stock] Fetching AAPL from 2023-01-01 to 2023-12-31 every 1 Day
[Stock] Fetching: 2023-01-01 to 2023-02-01
[Stock] Fetching: 2023-02-01 to 2023-03-01
[Stock] Fetching: 2023-03-01 to 2023-04-01
[Stock] Fetching: 2023-04-01 to 2023-05-01
[Stock] Fetching: 2023-05-01 to 2023-06-01
[Stock] Fetching: 2023-06-01 to 2023-07-01
[Stock] Fetching: 2023-07-01 to 2023-08-01
[Stock] Fetching: 2023-08-01 to 2023-09-01
[Stock] Fetching: 2023-09-01 to 2023-10-01
[Stock] Fetching: 2023-10-01 to 2023-11-01
[Stock] Fetching: 2023-11-01 to 2023-12-01
[Stock] Fetching: 2023-12-01 to 2024-01-01
Saved: AAPLDAILY2023Q1_chunk_1.csv (250 rows)
